## Data extraction and preprocessing

In [1]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.3 MB/s eta 0:00:00


In [2]:
import datasets

dataset = datasets.load_dataset('sms_spam')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [4]:
data = dataset['train']

In [5]:
import pandas as pd

data_dict = data.to_dict()

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data_dict)

df

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...
5569,This is the 2nd time we have tried 2 contact u...,1
5570,Will ü b going to esplanade fr home?\n,0
5571,"Pity, * was in mood for that. So...any other s...",0
5572,The guy did some bitching but I acted like i'd...,0


In [6]:
# Splitting the DataFrame into features and target variable
X = df.drop(columns=['label'])  # Replace 'target_column' with the name of your target column
y = df['label']

# Calculating the index to split the DataFrame
split_index = int(len(df) * 0.80)

# Splitting the data into training and testing sets
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

In [7]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [8]:
train_df

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...
4454,Or I guess &lt;#&gt; min\n,0
4455,I'm home. Ard wat time will u reach?\n,0
4456,"Storming msg: Wen u lift d phne, u say ""HELLO""...",0
4457,If you want to mapquest it or something look u...,0


In [9]:
test_df

,sms,label
4459,Die... I accidentally deleted e msg i suppose ...,0
4460,Welcome to UK-mobile-date this msg is FREE giv...,1
4461,This is wishing you a great day. Moji told me ...,0
4462,Thanks again for your reply today. When is ur ...,0
4463,"Sorry I flaked last night, shit's seriously go...",0
...,...,...
5569,This is the 2nd time we have tried 2 contact u...,1
5570,Will ü b going to esplanade fr home?\n,0
5571,"Pity, * was in mood for that. So...any other s...",0
5572,The guy did some bitching but I acted like i'd...,0


In [10]:
import re
import string
import pandas as pd
import numpy as np
import nltk
!pip install pyspellchecker
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [11]:
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def remove_punctuation(text):
    # Remove punctuation using regex
    return re.sub(r'[^\w\s]', '', text)

def preprocess_text_advanced(text):
    # Remove punctuation
    text = remove_punctuation(text)
    # Convert text to lowercase
    text = text.lower()
    # Handle URLs
    text = re.sub(r'http\S+|www\S+', 'url', text)
    # Handle email addresses
    text = re.sub(r'\S+@\S+', 'email', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

# preprocessing steps to the "sms" column of the train_df DataFrame
train_df['clean_sms'] = train_df['sms'].apply(preprocess_text_advanced)

# preprocessing steps to the "sms" column of the test_df DataFrame
test_df['clean_sms'] = test_df['sms'].apply(preprocess_text_advanced)


In [12]:
train_df

,sms,label,clean_sms
0,"Go until jurong point, crazy.. Available only ...",0,go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...\n,0,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,0,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",0,nah dont think go usf life around though
...,...,...,...
4454,Or I guess &lt;#&gt; min\n,0,guess ltgt min
4455,I'm home. Ard wat time will u reach?\n,0,im home ard wat time u reach
4456,"Storming msg: Wen u lift d phne, u say ""HELLO""...",0,storming msg wen u lift phne u say hello u knw...
4457,If you want to mapquest it or something look u...,0,want mapquest something look usf dogwood drive...


In [13]:
test_df

,sms,label,clean_sms
4459,Die... I accidentally deleted e msg i suppose ...,0,die accidentally deleted e msg suppose 2 put e...
4460,Welcome to UK-mobile-date this msg is FREE giv...,1,welcome ukmobiledate msg free giving free call...
4461,This is wishing you a great day. Moji told me ...,0,wishing great day moji told offer always speec...
4462,Thanks again for your reply today. When is ur ...,0,thanks reply today ur visa coming r u still bu...
4463,"Sorry I flaked last night, shit's seriously go...",0,sorry flaked last night shit seriously goin ro...
...,...,...,...
5569,This is the 2nd time we have tried 2 contact u...,1,2nd time tried 2 contact u u 750 pound prize 2...
5570,Will ü b going to esplanade fr home?\n,0,ü b going esplanade fr home
5571,"Pity, * was in mood for that. So...any other s...",0,pity mood soany suggestion
5572,The guy did some bitching but I acted like i'd...,0,guy bitching acted like id interested buying s...


In [14]:
ham_messages = train_df[train_df['label'] == 0]['clean_sms']
spam_messages = train_df[train_df['label'] == 1]['clean_sms']

In [15]:
# Calculate the total number of messages
total_messages = len(train_df)

# Calculate the probability of spam
probability_of_spam = len(spam_messages) / total_messages

# Print the probability of spam
print(f"Probability of spam: {probability_of_spam:.2%}")


Probability of spam: 13.50%


In [16]:
def count_vectorizer(corpus, binary=True):
    # Create a vocabulary from the corpus
    vocabulary = set()
    for message in corpus:
        vocabulary.update(message.split())

    # Map each word to an index in the vocabulary
    word_to_index = {word: index for index, word in enumerate(sorted(vocabulary))}

    # Initialize an empty matrix to store the binary vector representations
    binary_vectors = []
    for message in corpus:
        # Initialize a binary vector for the current message
        binary_vector = [0] * len(vocabulary)
        # Update the binary vector based on the presence of words in the message
        for word in message.split():
            if word in word_to_index:
                binary_vector[word_to_index[word]] = 1 if binary else 1  # If binary is False, use count instead of binary
        binary_vectors.append(binary_vector)

    # Create a DataFrame from the binary vectors
    df = pd.DataFrame(binary_vectors, columns=sorted(vocabulary))
    df['label'] = np.nan  # Initialize label column with NaN values
    return df

In [17]:
# Combine messages from train and test datasets
combined_messages = pd.concat([train_df['clean_sms'], test_df['clean_sms']])

# Create the count vectorizer using the combined messages
combined_vectorizer = count_vectorizer(combined_messages)

# Get labels from train and test datasets
combined_labels = pd.concat([train_df['label'], test_df['label']])

# Assign labels to the combined vectorized messages
combined_vectorizer['label'] = combined_labels

# Separate the vectorized messages back into train and test datasets
train_df_vectorized = combined_vectorizer[:len(train_df)]
test_df_vectorized = combined_vectorizer[len(train_df):]


In [18]:
train_df_vectorized

,0,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,...,zindgi,zoe,zogtorius,zoom,zouk,zyada,é,ü,üll,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
test_df_vectorized

,0,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,...,zindgi,zoe,zogtorius,zoom,zouk,zyada,é,ü,üll,label
4459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Naive Bayes

In [20]:
total_messages = len(train_df_vectorized)
num_spam_messages = train_df_vectorized['label'].sum()
num_ham_messages = total_messages - num_spam_messages
p_spam = num_spam_messages / total_messages
p_ham = num_ham_messages / total_messages

In [21]:
num_spam_messages

602

In [22]:
p_ham

0.8649921507064364

In [23]:
spam_messages_vectorized = train_df_vectorized[train_df_vectorized['label'] == 1].drop(columns=['label'])
ham_messages_vectorized = train_df_vectorized[train_df_vectorized['label'] == 0].drop(columns=['label'])

# Calculate the conditional probabilities p(xi | y) for spam and ham classes
p_word_given_spam = (spam_messages_vectorized.sum() + 1) / (num_spam_messages + 2)  # Laplace smoothing
p_word_given_ham = (ham_messages_vectorized.sum() + 1) / (num_ham_messages + 2)  # Laplace smoothing

In [24]:
spam_messages_vectorized.sum()

0               1
008704050406    2
0089my          1
0121            0
01223585236     1
               ..
zouk            1
zyada           0
é               0
ü               0
üll             0
Length: 8859, dtype: int64

In [25]:
p_word_given_spam

0               0.003311
008704050406    0.004967
0089my          0.003311
0121            0.001656
01223585236     0.003311
                  ...   
zouk            0.003311
zyada           0.001656
é               0.001656
ü               0.001656
üll             0.001656
Length: 8859, dtype: float64

In [26]:
print(p_word_given_ham.shape, train_df_vectorized.shape)

(8859,) (4459, 8860)


In [29]:
def predict_class_for_message(message_vector, p_spam, p_ham, p_word_given_spam, p_word_given_ham):
    # Calculate the probability of the message being spam
    spam_probability = np.prod((p_word_given_spam ** message_vector) * ((1 - p_word_given_spam) ** (1 - message_vector))) * p_spam
    # Calculate the probability of the message being ham
    ham_probability = np.prod((p_word_given_ham ** message_vector) * ((1 - p_word_given_ham) ** (1 - message_vector))) * p_ham
    if spam_probability > ham_probability:
        return 1  # Spam
    else:
        return 0  # Ham


In [30]:
predicted_labels = train_df_vectorized.apply(lambda row: predict_class_for_message(row.drop('label'), p_spam, p_ham, p_word_given_spam, p_word_given_ham), axis=1)

# Calculate accuracy
accuracy = (predicted_labels == train_df_vectorized['label']).mean()
print("Accuracy on train_df_vectorized:", accuracy)

Accuracy on train_df_vectorized: 0.989683785602153


In [31]:
predicted_labels_test = test_df_vectorized.apply(lambda row: predict_class_for_message(row.drop('label'), p_spam, p_ham, p_word_given_spam, p_word_given_ham), axis=1)

# Calculate accuracy
accuracy = (predicted_labels_test == test_df_vectorized['label']).mean()
print("Accuracy on test_df_vectorized:", accuracy)

Accuracy on test_df_vectorized: 0.9802690582959641


In [32]:
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
confusion_matrix = confusion_matrix(train_df_vectorized['label'], predicted_labels)

# Print the confusion matrix
print(confusion_matrix)


[[3855    2]
 [  44  558]]


## SVM

In [33]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize SVM classifier
svm_classifier = SVC(kernel='linear')

# Train the SVM classifier on the vectorized training data
svm_classifier.fit(train_df_vectorized.drop('label', axis=1), train_df_vectorized['label'])

# Predict the labels for the test_df_vectorized data
predicted_labels_test_svm = svm_classifier.predict(test_df_vectorized.drop('label', axis=1))

# Calculate the accuracy of the SVM classifier on the test_df_vectorized data
accuracy_svm = accuracy_score(test_df_vectorized['label'], predicted_labels_test_svm)
print("Accuracy of SVM classifier on test data:", accuracy_svm)

Accuracy of SVM classifier on test data: 0.9829596412556054


## DT

In [34]:
import numpy as np

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTree:
    def __init__(self, max_depth=None, max_features=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.root = None

    def fit(self, X, y):
        self.root = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        if (self.max_depth is not None and depth >= self.max_depth) or n_labels == 1 or n_samples < 2:
            return Node(value=self._most_common_label(y))

        # Random feature selection
        if self.max_features is not None:
            features = np.random.choice(n_features, self.max_features, replace=False)
        else:
            features = range(n_features)

        # Find the best split among selected features
        best_gain = 0
        best_feature = None
        best_threshold = None
        for feature in features:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self._information_gain(X, y, feature, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        if best_gain == 0:
            return Node(value=self._most_common_label(y))
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices
        left_child = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_child = self._grow_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)

    def _most_common_label(self, y):
        return np.bincount(y).argmax()

    def _information_gain(self, X, y, feature, threshold):
        parent_entropy = self._entropy(y)
        left_indices = X[:, feature] <= threshold
        right_indices = ~left_indices
        if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
            return 0
        left_entropy = self._entropy(y[left_indices])
        right_entropy = self._entropy(y[right_indices])
        child_entropy = (np.sum(left_indices) / len(y)) * left_entropy + (np.sum(right_indices) / len(y)) * right_entropy
        return parent_entropy - child_entropy

    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def predict(self, X):
        return np.array([self._predict_single(sample, self.root) for sample in X])

    def _predict_single(self, sample, node):
        if node.value is not None:
            return node.value
        if sample[node.feature] <= node.threshold:
            return self._predict_single(sample, node.left)
        else:
            return self._predict_single(sample, node.right)


In [35]:
# Initialize and train Decision Tree classifier
dt_classifier_scratch = DecisionTree(max_depth=36, max_features=None)
X_train_scratch = train_df_vectorized.drop('label', axis=1).values
y_train_scratch = train_df_vectorized['label'].values
dt_classifier_scratch.fit(X_train_scratch, y_train_scratch)

# Predict the labels for the test data
X_test_scratch = test_df_vectorized.drop('label', axis=1).values
predicted_labels_test_dt_scratch = dt_classifier_scratch.predict(X_test_scratch)

# Calculate the accuracy of the Decision Tree classifier
accuracy_dt_scratch = accuracy_score(test_df_vectorized['label'], predicted_labels_test_dt_scratch)
print("Accuracy of Decision Tree classifier (from scratch) on test data:", accuracy_dt_scratch)


Accuracy of Decision Tree classifier (from scratch) on test data: 0.9560538116591928


## Logistic Regression

In [36]:
import numpy as np
class LogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=100):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.max_iter):
            z = np.dot(X, self.weights) + self.bias
            y_pred = 1 / (1 + np.exp(-z))
            gradient_weights = -np.dot(X.T, (y - y_pred))
            gradient_bias = -np.sum(y - y_pred)

            self.weights -= self.learning_rate * gradient_weights
            self.bias -= self.learning_rate * gradient_bias

    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        y_pred = 1 / (1 + np.exp(-z))
        return np.where(y_pred > 0.5, 1, 0)


In [43]:
# Initialize and train Logistic Regression classifier
lr_classifier = LogisticRegression(learning_rate=0.015, max_iter=450)
X_train = train_df_vectorized.drop('label', axis=1).values
y_train = train_df_vectorized['label'].values
lr_classifier.fit(X_train, y_train)

# Predict the labels for the test data
X_test = test_df_vectorized.drop('label', axis=1).values
predicted_labels_test_lr = lr_classifier.predict(X_test)

# Calculate the accuracy of the Logistic Regression classifier
accuracy_lr = accuracy_score(test_df_vectorized['label'], predicted_labels_test_lr)
print("Accuracy of Logistic Regression classifier (from scratch) on test data:", accuracy_lr)

Accuracy of Logistic Regression classifier (from scratch) on test data: 0.9838565022421525


### final results updated set

In [38]:
import os

test_directory = "test"
if not os.path.exists(test_directory):
    os.makedirs(test_directory)

for index, row in test_df.iterrows():
    email_text = row['sms']
    email_filename = f"email{index + 1}.txt"
    with open(os.path.join(test_directory, email_filename), "w") as file:
        file.write(email_text)

print("Test emails have been saved to the 'test' directory.")


Test emails have been saved to the 'test' directory.


In [44]:
folder_path = "test"
for i, filename in enumerate(os.listdir(folder_path)):
    if filename.startswith("email") and filename.endswith(".txt"):
        # Update label in the email folder based on predicted label
        new_label = predicted_labels_test_lr[i]
        # Your code to update labels in the email folder goes here
        print(f"Updated label for {filename}: {new_label}")

Updated label for email5171.txt: 0
Updated label for email5424.txt: 1
Updated label for email4742.txt: 0
Updated label for email4879.txt: 0
Updated label for email4567.txt: 0
Updated label for email4580.txt: 0
Updated label for email5008.txt: 0
Updated label for email4875.txt: 0
Updated label for email4757.txt: 0
Updated label for email5235.txt: 0
Updated label for email5223.txt: 0
Updated label for email5417.txt: 0
Updated label for email5515.txt: 0
Updated label for email5033.txt: 0
Updated label for email5494.txt: 0
Updated label for email4878.txt: 0
Updated label for email5198.txt: 0
Updated label for email5256.txt: 0
Updated label for email4973.txt: 0
Updated label for email4775.txt: 0
Updated label for email4754.txt: 0
Updated label for email5366.txt: 0
Updated label for email4537.txt: 0
Updated label for email5526.txt: 0
Updated label for email5342.txt: 0
Updated label for email4597.txt: 0
Updated label for email4556.txt: 0
Updated label for email4625.txt: 0
Updated label for em